In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
import sys
import pudl

from pudl.settings import (Ferc1Settings, Ferc1DbfSettings, Ferc1XbrlSettings)
pudl_settings = pudl.workspace.setup.get_defaults()
from pudl.transform.ferc1 import *
from typing import Literal, Tuple

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

# XBRL

In [ ]:
ferc1_settings = Ferc1Settings(ferc1_xbrl_settings=Ferc1XbrlSettings(tables=[
    "steam_electric_generating_plant_statistics_large_plants_fuel_statistics_402_duration",
    "steam_electric_generating_plant_statistics_large_plants_402_duration",
    "steam_electric_generating_plant_statistics_large_plants_402_instant",
]
), ferc1_dbf_settings=Ferc1DbfSettings(),)

In [ ]:
ferc1_dbf_raw_dfs = pudl.extract.ferc1.extract_dbf(
    ferc1_settings=ferc1_settings.ferc1_dbf_settings, pudl_settings=pudl_settings
)

In [ ]:
# Extract FERC form 1 XBRL data
ferc1_xbrl_raw_dfs = pudl.extract.ferc1.extract_xbrl(
    ferc1_settings=ferc1_settings.ferc1_xbrl_settings, pudl_settings=pudl_settings
)

ferc1_transformed_dfs = {}

In [ ]:
rename_dict = {
    "plants_steam_ferc1_duration": 'steam_electric_generating_plant_statistics_large_plants_402_duration', 
    "plants_steam_ferc1_instant":'steam_electric_generating_plant_statistics_large_plants_402_instant',
    "fuel_ferc1_duration":'steam_electric_generating_plant_statistics_large_plants_fuel_statistics_402_duration'}
for pudl_name, xbrl_name in rename_dict.items():
    ferc1_xbrl_raw_dfs[pudl_name] = ferc1_xbrl_raw_dfs.pop(xbrl_name)

In [ ]:
table = "fuel_ferc1"
fuel_trans = FuelFerc1(table_name=table)
fuel_df = fuel_trans.execute(
    raw_dbf=ferc1_dbf_raw_dfs.get(table),
    raw_xbrl_instant=ferc1_xbrl_raw_dfs.get(f"{table}_instant", None),
    raw_xbrl_duration=ferc1_xbrl_raw_dfs.get(f"{table}_duration", None),
)

table="plants_steam_ferc1"
steam_df = PlantsSteamFerc1(
    table_name=table
).execute(
    raw_dbf=ferc1_dbf_raw_dfs.get(table),
    raw_xbrl_instant=ferc1_xbrl_raw_dfs.get(f"{table}_instant", None),
    raw_xbrl_duration=ferc1_xbrl_raw_dfs.get(f"{table}_duration", None),
    transformed_fuel=fuel_df,
)